# [Hands-On] Fine-tuning LLaMA for Translation (ENG-KOR): A Practical Guide

- Author: Sangkeun Jung (hugmanskj@gmail.com)

> Educational Purpose

**Copyright**: All rights reserved

## Overview

This notebook demonstrates how to fine-tune LLaMA 2 for English-to-Korean translation tasks using a parallel corpus dataset. We'll cover the complete process from data preparation to model evaluation, using efficient training techniques like LoRA (Low-Rank Adaptation) and 4-bit quantization.

Key learning objectives:
* Understanding how to prepare parallel corpus data for instruction fine-tuning
* Implementing efficient fine-tuning using LoRA
* Working with quantized models for memory efficiency
* Evaluating fine-tuned models on translation tasks


## Import Required Libraries

## Required Libraries

We'll import various libraries that are essential for our fine-tuning process:

- `torch`: The core deep learning framework
- `datasets`: For handling the SQuAD dataset
- `transformers`: Provides the model and tokenizer
- `peft`: For implementing LoRA
- `trl`: Contains the SFTTrainer for supervised fine-tuning
- Other utility libraries for data manipulation

These imports set up the foundation for our fine-tuning pipeline.

In [1]:
!pip install trl==0.12.0


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import os
import torch
from datasets import Dataset, load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd

## Data Preparation with Korean Parallel Corpus

### Understanding the Dataset
The Korean Parallel Corpus dataset contains:
- English sentences (source)
- Corresponding Korean translations (target)
- A diverse range of topics and sentence structures



In [3]:
from datasets import load_dataset

dataset = load_dataset("Moo/korean-parallel-corpora")
train_df = dataset['train'].to_pandas()
validation_df = dataset['validation'].to_pandas()
test_df = dataset['test'].to_pandas()

README.md:   0%|          | 0.00/297 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

dev.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/96215 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ko', 'en'],
        num_rows: 96215
    })
    validation: Dataset({
        features: ['ko', 'en'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['ko', 'en'],
        num_rows: 2000
    })
})

In [5]:
train_df.iloc[:10]

ko  \
0               개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"   
1  모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하...   
2                         그러나 이것은 또한 책상도 필요로 하지 않는다.   
3  79.95달러하는 이 최첨단 무선 광마우스는 허공에서 팔목, 팔, 그외에 어떤 부분...   
4  정보 관리들은 동남 아시아에서의 선박들에 대한 많은 (테러) 계획들이 실패로 돌아갔...   
5  이 지역에 있는 미국 선박과 상업용 선박들에 대한 알카에다의 (테러) 시도 중 여러...   
6  국립 과학 학회가 발표한 새 보고서에따르면, 복잡한 임무를 수행해야 하는 군인들이나...   
7  이 보고서에따르면, "특히, 군사 작전에서 생사가 걸린 상황이 될 수도 있는 반응 ...   
8          "결정적인 순간에 그들의 능력을 증가시켜 줄 그 무엇이 매우 중요합니다."   
9  연구가들이 이미 커피 대체품으로서 음식 대용 과자나 껌에 카페인을 첨가하는 방법을 ...   

                                                  en  
0  Much of personal computing is about "can you t...  
1  so a mention a few weeks ago about a rechargea...  
2  Like all optical mice, But it also doesn't nee...  
3  uses gyroscopic sensors to control the cursor ...  
4  Intelligence officials have revealed a spate o...  
5  After learning of several foiled al Qaeda atte...  
6  Caffeine can help increase reaction time and i...  
7  "Specifically, it can be used in maintaining s...  
8  "Something that will boost their capabilities ...  
9  Researchers are already exploring ways to put ...

### Data Format
We'll convert our parallel sentences into an instruction format suitable for LLaMA 2. Each example will contain:
1. English sentence: The source text to be translated
2. Korean sentence: The target translation

### Instruction Format Example:
```text
[INST]
Translate the following English sentence into Korean.
### Sentence
{english_text}
[/INST]
{korean_text}
```

In [6]:
# Define the conversion function
def convert_to_instruction_format(row):
    return f"""
    <s>
    [INST]
    Translate the following English sentence into Korean.
    ### Sentence
    {row['en']}
    [/INST]
    {row['ko']}
    </s>"""

# Apply the function to the DataFrame
train_df['text'] = train_df.apply(convert_to_instruction_format, axis=1)
validation_df['text'] = validation_df.apply(convert_to_instruction_format, axis=1)
test_df['text'] = validation_df.apply(convert_to_instruction_format, axis=1)

In [7]:
print( train_df.iloc[0].text )


    <s>
    [INST]
    Translate the following English sentence into Korean.
    ### Sentence
    Much of personal computing is about "can you top this?"
    [/INST]
    개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"
    </s>


In [8]:
train_df = train_df.iloc[:3000]

In [9]:
dataset = Dataset.from_pandas(train_df[['text']])
dataset

Dataset({
    features: ['text'],
    num_rows: 3000
})

## Model Configuration and Setup

### Base Model Selection
We're using the LLaMA 2 7B Chat model (`NousResearch/Llama-2-7b-chat-hf`) as our foundation. This model is:
- Optimized for dialogue and instruction following
- Pre-trained on a large corpus of text
- Suitable for fine-tuning on specific tasks

### Quantization Setup
To make training feasible on consumer GPUs, we implement 4-bit quantization:

```python
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)
```

In [10]:
model_name = "NousResearch/Llama-2-7b-chat-hf"

### Quantization Configuration

To reduce memory usage and enable training on consumer GPUs, we implement 4-bit quantization:

In [11]:
use_4bit = True  # Enable 4-bit precision
bnb_4bit_compute_dtype = "float16"  # Data type for computations
bnb_4bit_quant_type = "nf4"  # Quantization method
use_nested_quant = False  # Enable double quantization for more compression

#Setting up the data type for computation based on the precision setting
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

#Configuring the 4-bit quantization and precision for the model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Verifying if the current GPU supports bfloat16 to suggest using it for better performance
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Accelerate training with bf16=True")
        print("=" * 80)

Accelerate training with bf16=True


### LoRA Configuration Section

LoRA is a parameter-efficient fine-tuning technique that significantly reduces memory requirements while maintaining performance.



In [12]:
#LoRA settings for modifying attention mechanisms
lora_r = 64  # Dimension of LoRA attention
lora_alpha = 16  # Scaling factor for LoRA
lora_dropout = 0.1  # Dropout rate in LoRA layers

#Configuring LoRA parameters for the model to fine-tune its attention mechanisms
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",  # Setting the bias option for LoRA
    task_type="CAUSAL_LM",  # Defining the task type as causal language modeling
)

### Model and Tokenizer Initialization

Load and configure the model and tokenizer:



In [13]:
device_map = {"": 0}  # Load model on specific GPU
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False  # Disable caching to save memory
model.config.pretraining_tp = 1  # Setting pre-training task parallelism

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # Setting the pad token
tokenizer.padding_side = "right"  # Adjusting padding to the right to avoid issues during training


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

### Training Configuration

Set up training parameters:



In [14]:
training_arguments = TrainingArguments(
    output_dir="./results",

    ## epoch & gradient accumulation
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,

    ## optimzer
    optim="paged_adamw_32bit",
    weight_decay=0.001,
    max_grad_norm=0.3,

    ## Learning weight Schedule
    learning_rate=2e-4,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",

    ## Loging
    save_steps=0,
    logging_steps=25,

    ## Datatype
    fp16=False,
    bf16=False,

    ## Data Handling
    group_by_length=True,  # Grouping by length for efficient batching

    report_to=[],        # 모든 외부 로깅 비활성화 (TensorBoard, wandb 등)
)

### SFT training settings




In [15]:
max_seq_length = None  # Max sequence length
packing = False  # Pack short sequences together

#Setting up the fine-tuning trainer with the specified model, dataset, tokenizer, and training arguments
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,

    ## data handling
    train_dataset=dataset,
    dataset_text_field="text",  # Specifying which dataset field to use for text
    max_seq_length=max_seq_length,  # Setting the maximum sequence length
    packing=packing,  # Enabling packing for efficiency

    ## peft
    peft_config=peft_config,

    ## training args
    args=training_arguments,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## Training
Execute the training:

In [16]:
trainer.train()

TrainOutput(global_step=188, training_loss=1.0857811988668238, metrics={'train_runtime': 930.4038, 'train_samples_per_second': 3.224, 'train_steps_per_second': 0.202, 'total_flos': 1.8732432705060864e+16, 'train_loss': 1.0857811988668238, 'epoch': 1.0})

## Model Evaluation

For testing, we need to format our prompts without the answers:

In [17]:
def convert_to_prompt_format(row):
    return f"""
    <s>
    [INST]
    Translate the following English sentence into Korean.
    ### Sentence
    {row['en']}
    [/INST]
    """

validation_df['prompt'] = validation_df.apply(convert_to_prompt_format, axis=1)

In [18]:
logging.set_verbosity(logging.CRITICAL)
prompt = validation_df.iloc[0].prompt

pipe = pipeline(task="text-generation",
                model=model,
                tokenizer=tokenizer, max_length=300)

result = pipe(prompt)

print("-"*50)
print(prompt)
print("-"*50)
print("[PREDICTED]")
print(result[0]['generated_text'].split("[/INST]")[1])
print("="*50)
print("[ACTUAL]")
print(validation_df.iloc[0]['ko'])

--------------------------------------------------

    <s>
    [INST]
    Translate the following English sentence into Korean.
    ### Sentence
    After keeping the world's most powerful supercomputer to themselves for a year, government researchers showed off the $110 million wonder and said it might help save the world from nuclear war.
    [/INST]
    
--------------------------------------------------
[PREDICTED]

    그들은 최대의 초고속 컴퓨터를 1년 동안 세계에 독점하고 있었다.
    
[ACTUAL]
세계 에서 가장 강력한 수퍼컴퓨터를 1년 동안이나 독점해 오던 정부의 연구학자들이 이 1억1천만 달러 짜리 경이로운 설비를 공개하며, 이것이 핵전쟁으로부터 세계를 구하는데 도움이 될지도 모른다고 말했다.


In [19]:
n=54
prompt = validation_df.iloc[n].prompt

pipe = pipeline(task="text-generation",
                model=model,
                tokenizer=tokenizer, max_length=300)

result = pipe(prompt)

print("-"*50)
print(prompt)
print("-"*50)
print("[PREDICTED]")
print(result[0]['generated_text'].split("[/INST]")[1])
print("="*50)
print("[ACTUAL]")
print(validation_df.iloc[n]['ko'])

--------------------------------------------------

    <s>
    [INST]
    Translate the following English sentence into Korean.
    ### Sentence
    As the global market for data security grows, says Stephen Walker, the stakes will be in the hundreds of billions of dollars.
    [/INST]
    
--------------------------------------------------
[PREDICTED]

    세계 데이터 보안 시장이 성장하면서 애당초 싸움의 가치가 수십억 달러에 이르게 될 것이라고 스티븐 워커는 말한다.
    
[ACTUAL]
자료 보안에 대한 세계 시장이 커져 감에따라 시장 규모는 수천억 달러가 될 것이라고 스티븐 워커는 말한다.


In [20]:
#Fine-tuned model name
fine_tuned_model = "Llama-2-7b-translation-chat-finetune"
#Save trained model
trainer.model.save_pretrained(fine_tuned_model)

## Conclusion

In this tutorial, we've covered:

1. How to prepare parallel corpus data for fine-tuning LLaMA 2
2. Implementation of efficient training techniques (LoRA and 4-bit quantization)
3. The complete fine-tuning pipeline for translation tasks
4. Model evaluation and inference

The fine-tuned model can now effectively translate English sentences to Korean, making it suitable for various translation applications.

Key takeaways:
- LoRA enables efficient fine-tuning by updating only a small number of parameters
- 4-bit quantization makes it possible to fine-tune large models on consumer hardware
- Proper prompt formatting is crucial for instruction fine-tuning
- The resulting model can be used in a pipeline for easy translation inference